In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [39]:
import datajoint as dj
import pandas as pd
import numpy as np
import subprocess
import numpy as np
import u19_pipeline.utils.path_utils as pu
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import datetime
from shutil import which

from u19_pipeline.utils.matlab_utils import convert_mat_file_to_dict, convert_function_handle_to_str
from scipy.io import loadmat, matlab

acquisition = dj.create_virtual_module('acquisition', 'u19_acquisition')


## Add MATLAB path to system PATH 

In [3]:
local_matlab_path = '/Applications/MATLAB_R2020a.app/bin/'
if which('matlab') is None:
    os.environ['PATH'] += ':'+local_matlab_path

In [4]:
if which('matlab') is None:
    print('Matlab command not found, add correct path installation or proceed (Choice &  TrialType data will not be read from the behavior file')
else:
    print('Matlab installation found !!')

Matlab installation found !!


In [54]:
#subject = 'jeremyjc_j016'
#date = '2021-11-21'

subject = 'efonseca_ef884_actp003'
date = '2021-12-03'

path = (acquisition.SessionStarted & dict(session_date =date, subject_fullname=subject)).fetch1('remote_path_behavior_file')
mat_file = '/Volumes' + path

In [55]:
 matin = convert_mat_file_to_dict(mat_file)
 converted_metadata = convert_function_handle_to_str(mat_file_path=mat_file)



File written
convert_function_to_txt.m


In [56]:
def convert_towers_block2_df(current_block_trial):
    
    valid_block = 0
    # "Normal" blocks are stored as numpy arrays and its length is greater than 0
    if isinstance(current_block_trial, np.ndarray) and current_block_trial.shape[0] > 0:
        current_block_trial = current_block_trial.tolist()
        valid_block = 1 
    # One trial blocks are stored as dictionaries
    if isinstance(current_block_trial, dict):
        current_block_trial = [current_block_trial]
        valid_block = 1 

    if valid_block:
        block_trial_df = pd.DataFrame(current_block_trial)
    else:
        block_trial_df = pd.DataFrame()

    return valid_block, block_trial_df





In [59]:

session_block_trial_df = pd.DataFrame()

#Convert all blocks to dataframe and append them

#For a single block sessions
if isinstance(matin['log']['block'], dict):
    print('here only one block')
    valid_block, block_df = convert_towers_block2_df(matin['log']['block']['trial'])
    if valid_block:
        session_block_trial_df = block_df.copy()
#For multiple blocks sessions
else:
    num_blocks_conv = 0
    for i in range(matin['log']['block'].shape[0]):
        
        #Try to convert block to dataframe and append to session DF
        valid_block, block_df = convert_towers_block2_df(matin['log']['block'][i]['trial'])
        print(valid_block)
        if valid_block:
            if num_blocks_conv == 0:
                session_block_trial_df = block_df.copy()
            else:
                session_block_trial_df = session_block_trial_df.append(block_df)
            num_blocks_conv +=1



here only one block


,choice,collision,comment,cueCombo,cueOffset,cueOnset,cuePos,duration,excessTravel,finalchoice,...,rewardScale,sensorDots,start,state_ports,time,trialID,trialProb,trialType,velocity,viStart
0,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[1, 1, 1, 1, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0], 0]","[[11648, 10692, 10909, 11149, 11235, 11289, 11...","[[194.57321, 25.120554, 53.8311, 100.35614, 11...",138.619217,0.117929,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, -1, -1, 52], [0, 0, -8, 0, 1312], [0, ...",1.313610,"[[[0.0, 0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1...","[0.0, 0.0403214, 0.0590853, 0.0766748, 0.12813...",1.0,0.5,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...",2
1,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], []]","[[1096, 1216, 1423, 158, 609, 659, 848, 959, 1...","[[138.80388, 181.81398, 198.20818, 26.643602, ...",49.268803,-0.022894,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, 0, 0, 8], [0, 0, 0, 0, 112], [0, 0, 0,...",139.932831,"[[[0.0, 0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1...","[0.0, 0.0222565, 0.0290751, 0.0419076, 0.04851...",2.0,0.5,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, ...",16504
2,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 1, ...","[[0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]","[[101, 1473], [94, 479, 540, 703, 742, 805, 10...","[[18.048697, 173.02887], [14.99978, 36.38115, ...",28.197702,-0.027195,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, -8, 0, 8], [0, 0, -77, 0, 84], [0, 0, ...",189.201630,"[[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 1.0, 1...","[0.0, 0.0212646, 0.0280473, 0.0352011, 0.04214...",3.0,0.5,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 17.13...",22360
3,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]]","[[0, 0, 0, 0, 0, 0], []]","[[1246, 1321, 1546, 325, 401, 776], []]","[[121.804306, 152.67021, 192.59846, 16.429367,...",24.094978,-0.018349,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, -20, -2, 12], [0, 0, -119, 5, 76], [0,...",217.399338,"[[[0.0, 0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1...","[0.0, 0.0211291, 0.0279587, 0.0345268, 0.04075...",-4.0,0.5,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 31.82...",25699
4,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, ...","[[], [0, 0, 0, 0, 0, 0, 0, 0]]","[[], [1110, 1302, 1359, 3053, 3146, 3222, 617,...","[[], [81.91047, 122.81532, 141.81816, 174.9878...",36.871433,0.103975,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, -5, -4, 12], [0, 0, -9, 9, 108], [0, 0...",241.494308,"[[[0.0, 0.0, 1.0, 1.0, 1.0], [0.0, 1.0, 1.0, 1...","[0.0, 0.0308128, 0.0398561, 0.047442, 0.055039...",4.0,0.5,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 1.082...",28539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[[1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0], []]","[[138, 183, 205, 228, 258, 287, 346, 529, 64],...","[[48.467873, 75.81385, 89.13162, 103.86104, 12...",9.990882,-0.055342,"[[b'', b'MCOS', b'Choice', <scipy.io.matlab.mi...",...,1.0,"[[0, 0, 0, 0, 8], [0, 0, 0, 0, 56], [0, 0, 0, ...",3

## Get storage by year

In [29]:
lo = pd.DataFrame([matin['log']['block'][1]['trial']])

In [44]:
type(matin['log']['block'][0]['trial'])

dict

## Get storage by main directories 

In [10]:
 len(converted_metadata['trial_type'])

186

In [15]:
import sys
sys.path.append('/Applications/MATLAB_R2020a.app/bin/')

In [16]:
sys.path

['/Users/alvaros/Documents/MATLAB/BrainCogsProjects/Datajoint_proj/U19-pipeline_python/notebooks',
 '/Users/alvaros/.vscode/extensions/ms-python.python-2020.9.114305/pythonFiles',
 '/Users/alvaros/.vscode/extensions/ms-python.python-2020.9.114305/pythonFiles/lib/python',
 '/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python37.zip',
 '/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python3.7',
 '/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python3.7/lib-dynload',
 '',
 '/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python3.7/site-packages',
 '/Users/alvaros/Documents/MATLAB/BrainCogsProjects/Datajoint_proj/U19-pipeline_python',
 '/Users/alvaros/opt/anaconda3/envs/u19_datajoint/lib/python3.7/site-packages/IPython/extensions',
 '/Users/alvaros/.ipython',
 '/Applications/MATLAB_R2020a.app/bin/']

In [18]:
import matlab.engine

ModuleNotFoundError: No module named 'matlab'

In [20]:
os.system('''
#!/bin/bash
matlab -nosplash -nodisplay 
''')

32512

In [23]:
os.environ

environ{'VSCODE_NODE_CACHED_DATA_DIR': '/Users/alvaros/Library/Application Support/Code/CachedData/db40434f562994116e5b21c24015a2e40b2504e6',
        'SHELL': '/bin/zsh',
        'AMD_ENTRYPOINT': 'vs/workbench/services/extensions/node/extensionHostProcess',
        'TMPDIR': '/var/folders/sg/5bw1t8p11nx09k7kmytnmfb40000gp/T/',
        'CONDA_SHLVL': '2',
        'CONDA_PROMPT_MODIFIER': '(u19_datajoint) ',
        'PYTHONUNBUFFERED': '1',
        'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined',
        'DJ_USER': 'alvaros',
        'PYTHONIOENCODING': 'utf-8',
        'USER': 'alvaros',
        'CONDA_EXE': '/Users/alvaros/opt/anaconda3/bin/conda',
        'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.mYt8DRqATB/Listeners',
        '__CF_USER_TEXT_ENCODING': '0x1F6:0x0:0x0',
        'VSCODE_LOG_STACK': 'false',
        'ELECTRON_RUN_AS_NODE': '1',
        '_CE_CONDA': '',
        'CONDA_PREFIX_1': '/Users/alvaros/opt/anaconda3',
        'VSCODE_LOGS': '/Users/alvaros/Library/Applicatio